### Hyper-parameter Tunning with Genetic Algorithm
#### Created: Dec. 29 2018

In [1]:
from pprint import pprint
import datetime
import os

In [2]:
import sys
sys.path.append("../")
# If this notebook file is not placed under in /notebook/ directory,
# adding directory "../" might not correly add the project directory.
# If adding "../" does not solve the importing problem, we need to setup 
# the directory mannually.
try:
    import constants
except ModuleNotFoundError:
    core_dir = input("Directory of core files >>> ")
    if not core_dir.endswith("/"):
        core_dir += "/"
    sys.path.append(core_dir)
    import constants

In [3]:
import core.genetic.control as control
import core.genetic.tuner as tuner
import core.models.stacked_lstm as stacked_lstm

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.24.1) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [4]:
# Load dataset.
print("Dataset paths saved:")
pprint(constants.DATA_DIR)
choice = None
while choice is None or choice not in constants.DATA_DIR.keys():
    if choice is not None:
        print("Invalid data location received, try again...")
    choice = input("Select Dataset >>> ")
FILE_DIR = constants.DATA_DIR[choice]
print(f"Dataset chosen: {FILE_DIR}")

Dataset paths saved:
{'a': '/Users/tianyudu/Documents/Academics/EconForecasting/AnnEconForecast/data/UNRATE.csv',
 'b': '/home/ec2-user/AnnEconForecast/data/UNRATE.csv',
 'c': '/home/ec2-user/AnnEconForecast/data/DEXCAUS.csv'}
Select Dataset >>> a
Dataset chosen: /Users/tianyudu/Documents/Academics/EconForecasting/AnnEconForecast/data/UNRATE.csv


In [5]:
# Load configuration.
print("Avaiable configuration files found: ")
for cf in os.listdir("../hps_configs"):
    if cf.endswith("config.py"):
        print("\t" + cf)

config_name = input("Select config file >>> ")
if config_name.endswith(".py"):
    config_name = config_name[:-3]
# config_name = "mac_config"

exec(f"import hps_configs.{config_name} as config")

Avaiable configuration files found: 
	ec2_config.py
	mac_config.py
Select config file >>> mac_config


In [12]:
# Training Spec.
GENE_POOL = config.main
TOTAL_GEN = 3
POP_SIZE = 5
IGNORE_SET = (
    "PERIODS", "ORDER", "TRAIN_RATIO", "VAL_RATIO", "num_outputs", "num_inputs", "report_periods",
    "tensorboard_path", "model_path", "fig_path"
)

In [13]:
def eval_net_wrapper(param):
    return control.eval_net(
        model=stacked_lstm.StackedLSTM,
        param=param,
        file_dir=FILE_DIR,
        metric="mse_val",
        smooth_metric=0.05
    )

In [26]:
# Create the Optimizer.
optimizer = tuner.GeneticTuner(
    gene_pool=config.main,
    pop_size=POP_SIZE,
    eval_func=eval_net_wrapper,
    mode="min",
    retain=0.5,
    shot_prob=0.05,
    mutate_prob=0.05,
    verbose=False,
    ignore=IGNORE_SET
)

In [15]:
start_time = datetime.datetime.now()

In [19]:
elite = control.train_op(
    optimizer=optimizer,
    total_gen=TOTAL_GEN,
    elite=0.5
)

Generation: [0/3]
Evaluating the initial population.
 #################### Evaluating population[5/5, 100.00%]Best fitted entity validatiton MSE:  0.0248224        
Worst fitted entity validation MSE:  0.8676306
Generation: [1/3]
 #################### Evaluating population[6/6, 100.00%]Best fitted entity validatiton MSE:  0.0252026        
Worst fitted entity validation MSE:  0.0288695
Generation: [2/3]
 #################### Evaluating population[5/5, 100.00%]Best fitted entity validatiton MSE:  0.0244793        
Worst fitted entity validation MSE:  0.0258422
Generation: [3/3]
 #################### Evaluating population[6/6, 100.00%]Best fitted entity validatiton MSE:  0.0245235        
Worst fitted entity validation MSE:  0.0279572
Final:
Best fitted entity validatiton MSE:  0.0245235        
Worst fitted entity validation MSE:  0.0279572


In [20]:
end_time = datetime.datetime.now()
print(f"Time taken: {end_time - start_time}")

Time taken: 0:05:31.526424


In [25]:
elite[list(elite.keys())[-1]][0]

{'PERIODS': 1,
 'ORDER': 1,
 'LAGS': 6,
 'TRAIN_RATIO': 0.8,
 'VAL_RATIO': 0.1,
 'epochs': 150,
 'num_inputs': 1,
 'num_outputs': 1,
 'num_time_steps': 6,
 'num_neurons': [16, 32],
 'learning_rate': 0.1,
 'clip_grad': None,
 'report_periods': 10,
 'tensorboard_path': '/Volumes/Intel/annef_model_data/2016DEC17_MAC_02/tensorboard/',
 'model_path': '/Volumes/Intel/annef_model_data/2016DEC17_MAC_02/saved_models/',
 'fig_path': '/Volumes/Intel/annef_model_data/2016DEC17_MAC_02/model_figs/'}